In [2]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Conv2D, ReLU, Conv2DTranspose, Concatenate, LeakyReLU, Dropout, Flatten, Dense
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

# Paths
base_path = r"/kaggle/input/sar-s1-s2/dataset2/v_2"
checkpoint_dir = r"/kaggle/working/Model"
os.makedirs(checkpoint_dir, exist_ok=True)

categories = ['agri', 'barrenland', 'greenland', 'urban']
s1_paths = [os.path.join(base_path, category, 's1') for category in categories]
s2_paths = [os.path.join(base_path, category, 's2') for category in categories]

# Data loading and preprocessing
def load_image(file_path, is_color=False):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_png(image, channels=3 if is_color else 1)
    image = tf.image.resize(image, [256, 256])
    image = (tf.cast(image, tf.float32) / 127.5) - 1.0  # Normalize to [-1, 1]
    return image

def process_pair(s1_path, s2_path):
    s1_image = load_image(s1_path, is_color=False)  # Grayscale
    s2_image = load_image(s2_path, is_color=True)   # Color
    return s1_image, s2_image

def create_dataset(s1_paths, s2_paths, batch_size=16, images_per_category=40):
    s1_files = []
    s2_files = []
    for s1_path, s2_path in zip(s1_paths, s2_paths):
        s1_category_files = sorted(tf.io.gfile.glob(os.path.join(s1_path, '*.png')))[:images_per_category]
        s2_category_files = sorted(tf.io.gfile.glob(os.path.join(s2_path, '*.png')))[:images_per_category]
        s1_files.extend(s1_category_files)
        s2_files.extend(s2_category_files)

    s1_files.sort()
    s2_files.sort()

    s1_dataset = tf.data.Dataset.from_tensor_slices(s1_files)
    s2_dataset = tf.data.Dataset.from_tensor_slices(s2_files)
    dataset = tf.data.Dataset.zip((s1_dataset, s2_dataset))
    dataset = dataset.map(lambda s1, s2: process_pair(s1, s2))
    dataset = dataset.shuffle(buffer_size=100)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset
dataset = create_dataset(s1_paths, s2_paths, batch_size=8)
print(len(dataset))


4


In [3]:
# # Build generator and discriminator
def build_generator(input_shape=(256, 256, 1)):
    inputs = Input(shape=input_shape)
    x = inputs

    # Encoder - Downsampling
    x1 = Conv2D(64, (3, 3), padding='same')(x)
    x1 = ReLU()(x1)
    x1 = Conv2D(64, (3, 3), padding='same')(x1)
    x1 = ReLU()(x1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(x1)

    x2 = Conv2D(128, (3, 3), padding='same')(p1)
    x2 = ReLU()(x2)
    x2 = Conv2D(128, (3, 3), padding='same')(x2)
    x2 = ReLU()(x2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(x2)

    x3 = Conv2D(256, (3, 3), padding='same')(p2)
    x3 = ReLU()(x3)
    x3 = Conv2D(256, (3, 3), padding='same')(x3)
    x3 = ReLU()(x3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(x3)

    x4 = Conv2D(512, (3, 3), padding='same')(p3)
    x4 = ReLU()(x4)
    x4 = Conv2D(512, (3, 3), padding='same')(x4)
    x4 = ReLU()(x4)
    p4 = tf.keras.layers.MaxPooling2D((2, 2))(x4)

    # Additional Layer 1
    x5 = Conv2D(1024, (3, 3), padding='same')(p4)
    x5 = ReLU()(x5)
    x5 = Conv2D(1024, (3, 3), padding='same')(x5)
    x5 = ReLU()(x5)
    p5 = tf.keras.layers.MaxPooling2D((2, 2))(x5)
    
    # Additional Layer 2
    x6 = Conv2D(2048, (3, 3), padding='same')(p5)
    x6 = ReLU()(x6)
    x6 = Conv2D(2048, (3, 3), padding='same')(x6)
    x6 = ReLU()(x6)
  
    
    # Decoder - Upsampling
    u1 = Conv2DTranspose(1024, (3, 3), strides=(2, 2), padding='same')(x6)
    u1 = Concatenate()([u1, x5])
    u1 = Conv2D(1024, (3, 3), padding='same')(u1)
    u1 = ReLU()(u1)
    u1 = Conv2D(1024, (3, 3), padding='same')(u1)
    u1 = ReLU()(u1)

    u2 = Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same')(u1)
    u2 = Concatenate()([u2, x4])
    u2 = Conv2D(512, (3, 3), padding='same')(u2)
    u2 = ReLU()(u2)
    u2 = Conv2D(512, (3, 3), padding='same')(u2)
    u2 = ReLU()(u2)

    u3 = Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(u2)
    u3 = Concatenate()([u3, x3])
    u3 = Conv2D(256, (3, 3), padding='same')(u3)
    u3 = ReLU()(u3)
    u3 = Conv2D(256, (3, 3), padding='same')(u3)
    u3 = ReLU()(u3)

    u4 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(u3)
    u4 = Concatenate()([u4, x2])
    u4 = Conv2D(128, (3, 3), padding='same')(u4)
    u4 = ReLU()(u4)
    u4 = Conv2D(128, (3, 3), padding='same')(u4)
    u4 = ReLU()(u4)

    u5 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(u4)
    u5 = Concatenate()([u5, x1])
    u5 = Conv2D(64, (3, 3), padding='same')(u5)
    u5 = ReLU()(u5)
    u5 = Conv2D(64, (3, 3), padding='same')(u5)
    u5 = ReLU()(u5)

    outputs = Conv2D(3, (1, 1), activation='tanh')(u5)  # Output in range [-1, 1]
    model = tf.keras.models.Model(inputs, outputs)
    return model

#build discriminator
def build_discriminator(input_shape=(256, 256, 3)):
    inputs = Input(shape=input_shape)
    
    x = Conv2D(64, (3, 3), padding='same', strides=(2, 2))(inputs)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)

    x = Conv2D(128, (3, 3), padding='same', strides=(2, 2))(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)

    x = Conv2D(256, (3, 3), padding='same', strides=(2, 2))(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)

    x = Conv2D(512, (3, 3), padding='same', strides=(2, 2))(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)

    # Added two more layers
    x = Conv2D(1024, (3, 3), padding='same', strides=(2, 2))(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)

    x = Conv2D(2048, (3, 3), padding='same', strides=(2, 2))(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)

    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs, x)
    return model



In [3]:
# # Build generator and discriminator
# def build_generator(input_shape=(256, 256, 1)):
#     inputs = Input(shape=input_shape)
#     x = inputs

#     x1 = Conv2D(64, (3, 3), padding='same')(x)
#     x1 = ReLU()(x1)
#     x1 = Conv2D(64, (3, 3), padding='same')(x1)
#     x1 = ReLU()(x1)
#     p1 = tf.keras.layers.MaxPooling2D((2, 2))(x1)

#     x2 = Conv2D(128, (3, 3), padding='same')(p1)
#     x2 = ReLU()(x2)
#     x2 = Conv2D(128, (3, 3), padding='same')(x2)
#     x2 = ReLU()(x2)
#     p2 = tf.keras.layers.MaxPooling2D((2, 2))(x2)

#     x3 = Conv2D(256, (3, 3), padding='same')(p2)
#     x3 = ReLU()(x3)
#     x3 = Conv2D(256, (3, 3), padding='same')(x3)
#     x3 = ReLU()(x3)
#     p3 = tf.keras.layers.MaxPooling2D((2, 2))(x3)

#     x4 = Conv2D(512, (3, 3), padding='same')(p3)
#     x4 = ReLU()(x4)
#     x4 = Conv2D(512, (3, 3), padding='same')(x4)
#     x4 = ReLU()(x4)

#     x5 = Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(x4)
#     x5 = Concatenate()([x5, x3])
#     x5 = Conv2D(256, (3, 3), padding='same')(x5)
#     x5 = ReLU()(x5)
#     x5 = Conv2D(256, (3, 3), padding='same')(x5)
#     x5 = ReLU()(x5)

#     x6 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(x5)
#     x6 = Concatenate()([x6, x2])
#     x6 = Conv2D(128, (3, 3), padding='same')(x6)
#     x6 = ReLU()(x6)
#     x6 = Conv2D(128, (3, 3), padding='same')(x6)
#     x6 = ReLU()(x6)

#     x7 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(x6)
#     x7 = Concatenate()([x7, x1])
#     x7 = Conv2D(64, (3, 3), padding='same')(x7)
#     x7 = ReLU()(x7)
#     x7 = Conv2D(64, (3, 3), padding='same')(x7)
#     x7 = ReLU()(x7)

#     outputs = Conv2D(3, (1, 1), activation='tanh')(x7)  # Output in range [-1, 1]
#     model = tf.keras.models.Model(inputs, outputs)
#     return model

# def build_discriminator(input_shape=(256, 256, 3)):
#     inputs = Input(shape=input_shape)
#     x = Conv2D(64, (3, 3), padding='same', strides=(2, 2))(inputs)
#     x = LeakyReLU(alpha=0.2)(x)
#     x = Dropout(0.3)(x)

#     x = Conv2D(128, (3, 3), padding='same', strides=(2, 2))(x)
#     x = LeakyReLU(alpha=0.2)(x)
#     x = Dropout(0.3)(x)

#     x = Conv2D(256, (3, 3), padding='same', strides=(2, 2))(x)
#     x = LeakyReLU(alpha=0.2)(x)
#     x = Dropout(0.3)(x)

#     x = Conv2D(512, (3, 3), padding='same', strides=(2, 2))(x)
#     x = LeakyReLU(alpha=0.2)(x)
#     x = Dropout(0.3)(x)

#     x = Flatten()(x)
#     x = Dense(1, activation='sigmoid')(x)
#     model = tf.keras.models.Model(inputs, x)
#     return model



In [4]:
# Custom loss functions
def gradient_penalty(discriminator, real_images, fake_images, batch_size):
    alpha = tf.random.uniform([batch_size, 1, 1, 1], minval=0., maxval=1.)
    interpolated_images = alpha * real_images + (1 - alpha) * fake_images
    with tf.GradientTape() as tape:
        tape.watch(interpolated_images)
        interpolated_output = discriminator(interpolated_images, training=True)
    gradients = tape.gradient(interpolated_output, interpolated_images)
    norm = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=[1, 2, 3]))
    penalty = tf.reduce_mean((norm - 1.0) ** 2)
    return penalty

def feature_matching_loss(real_images, generated_images, discriminator):
    real_features = discriminator.layers[-2](real_images, training=False)
    fake_features = discriminator.layers[-2](generated_images, training=False)
    return tf.reduce_mean(tf.square(real_features - fake_features))

def custom_generator_loss(real_images, generated_images, discriminator, lambda_feat=10.0):
    mse_loss = tf.keras.losses.MeanSquaredError()(real_images, generated_images)
    feat_loss = feature_matching_loss(real_images, generated_images, discriminator)
    return mse_loss + lambda_feat * feat_loss

def custom_discriminator_loss(real_output, fake_output, real_images, fake_images, batch_size, discriminator, lambda_gp=10.0, smooth_real=0.9, smooth_fake=0.1):
    real_labels = tf.ones_like(real_output) * smooth_real
    fake_labels = tf.zeros_like(fake_output) + smooth_fake
    
    real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(real_labels, real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(fake_labels, fake_output)
    
    gp = gradient_penalty(discriminator, real_images, fake_images, batch_size)
    return real_loss + fake_loss + lambda_gp * gp

def save_images(generator, dataset, epoch, path, num_samples=3):
    sample_images, real_targets = next(iter(dataset))
    
    # Adjust num_samples if fewer images are available
    batch_size = tf.shape(sample_images)[0]
    if batch_size < num_samples:
        num_samples = batch_size  # Adjust if there are fewer samples than num_samples

    generated_images = generator(sample_images, training=False)
    
    # Rescale images from [-1, 1] to [0, 1]
    generated_images = (generated_images + 1) / 2.0
    sample_images = (sample_images + 1) / 2.0
    real_targets = (real_targets + 1) / 2.0

    # Handle the case where num_samples is 1
    if num_samples == 1:
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    else:
        fig, axs = plt.subplots(num_samples, 3, figsize=(15, num_samples * 5))
    
    # Flatten axs for easy indexing
    axs = axs.flatten()

    for i in range(num_samples):
        # Extract images considering batch size and reshaping
        sample_image = sample_images[i].numpy().squeeze() if batch_size > 1 else sample_images[0].numpy().squeeze()
        real_target = real_targets[i].numpy().squeeze() if batch_size > 1 else real_targets[0].numpy().squeeze()
        generated_image = generated_images[i].numpy().squeeze() if batch_size > 1 else generated_images[0].numpy().squeeze()
        
        # Rescale images to [0, 1]
        sample_image = np.clip(sample_image, 0, 1)
        real_target = np.clip(real_target, 0, 1)
        generated_image = np.clip(generated_image, 0, 1)
        
        axs[3*i].imshow(sample_image, cmap='gray' if sample_image.shape[-1] == 1 else None)
        axs[3*i].set_title('Greyscale Image')
        axs[3*i].axis('off')

        axs[3*i + 1].imshow(real_target)
        axs[3*i + 1].set_title('Ground Truth')
        axs[3*i + 1].axis('off')

        axs[3*i + 2].imshow(generated_image)
        axs[3*i + 2].set_title('Generated Image')
        axs[3*i + 2].axis('off')

    plt.tight_layout()
    plt.savefig(os.path.join(path, f'images_epoch_{epoch+1}.png'))
    plt.close()



In [5]:
# Function to save model and optimizer states
def save_checkpoint(generator, discriminator, generator_optimizer, discriminator_optimizer, checkpoint_dir, epoch):
    # Save model weights
    generator.save_weights(os.path.join(checkpoint_dir, 'generator_weights.weights.h5'))
    discriminator.save_weights(os.path.join(checkpoint_dir, 'discriminator_weights.weights.h5'))
    
    # Save optimizer states
    tf.saved_model.save(generator_optimizer, os.path.join(checkpoint_dir, 'generator_optimizer'))
    tf.saved_model.save(discriminator_optimizer, os.path.join(checkpoint_dir, 'discriminator_optimizer'))

# Function to load model and optimizer states
def load_checkpoint(generator, discriminator, generator_optimizer, discriminator_optimizer, checkpoint_dir):
    if os.path.exists(os.path.join(checkpoint_dir, 'generator_weights.h5')):
        # Load model weights
        generator.load_weights(os.path.join(checkpoint_dir, 'generator_weights.h5'))
        discriminator.load_weights(os.path.join(checkpoint_dir, 'discriminator_weights.h5'))

        # Load optimizer states
        generator_optimizer = tf.saved_model.load(os.path.join(checkpoint_dir, 'generator_optimizer'))
        discriminator_optimizer = tf.saved_model.load(os.path.join(checkpoint_dir, 'discriminator_optimizer'))
        
    return generator_optimizer, discriminator_optimizer

In [6]:
# Training loop
def train(generator, discriminator, dataset, epochs, checkpoint_dir, batch_size=32, lambda_feat=10.0, lambda_gp=10.0):
    generator_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
    discriminator_optimizer = Adam(learning_rate=0.0004, beta_1=0.5)
    # Load from checkpoint if available
#     generator_optimizer, discriminator_optimizer = load_checkpoint(generator, discriminator, generator_optimizer, discriminator_optimizer, checkpoint_dir)
    for epoch in range(epochs):
        for real_images, real_targets in tqdm(dataset):
            batch_size = real_images.shape[0]

            with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
                generated_images = generator(real_images, training=True)
                real_output = discriminator(real_targets, training=True)
                fake_output = discriminator(generated_images, training=True)

                # Generator loss
                gen_loss = custom_generator_loss(real_targets, generated_images, discriminator, lambda_feat)

                # Discriminator loss
                disc_loss = custom_discriminator_loss(real_output, fake_output, real_targets, generated_images, batch_size, discriminator, lambda_gp)

            generator_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
            discriminator_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

            generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
            discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))
        
        if (epoch+1)%10==0:
#             # Save checkpoint and sample images every epoch
            generator.save(os.path.join(checkpoint_dir, f'generator.h5'))
            discriminator.save(os.path.join(checkpoint_dir, f'discriminator.h5'))
#             save_checkpoint(generator, discriminator, generator_optimizer, discriminator_optimizer, checkpoint_dir, epoch)
            # Save generated images to monitor progress
            save_images(generator, dataset, epoch, checkpoint_dir)

        print(f"Epoch [{epoch + 1}/{epochs}] completed. Generator loss: {gen_loss.numpy()}, Discriminator loss: {disc_loss.numpy()}")

# Initialize and run training
generator = build_generator()
discriminator = build_discriminator()


train(generator, discriminator, dataset, epochs=500, checkpoint_dir=checkpoint_dir)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
  0%|          | 0/4 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/keras/src/backend/tensorflow/nn.py:674: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
100%|██████████| 4/4 [00:20<00:00,  5.16s/it]


Epoch [1/500] completed. Generator loss: 3.4572365283966064, Discriminator loss: 10.963432312011719


100%|██████████| 4/4 [00:03<00:00,  1.08it/s]


Epoch [2/500] completed. Generator loss: 2.7553768157958984, Discriminator loss: 10.823015213012695


100%|██████████| 4/4 [00:03<00:00,  1.08it/s]


...

Epoch [498/500] completed. Generator loss: 0.09993498772382736, Discriminator loss: nan


100%|██████████| 4/4 [00:03<00:00,  1.09it/s]


Epoch [499/500] completed. Generator loss: 0.07640957832336426, Discriminator loss: nan


100%|██████████| 4/4 [00:03<00:00,  1.06it/s]


Epoch [500/500] completed. Generator loss: 0.10347429662942886, Discriminator loss: nan


In [7]:
# from tensorflow.keras.models import load_model
# import numpy as np
# import cv2
# import matplotlib.pyplot as plt

# # Load the trained generator model
# generator = load_model('/kaggle/working/Model/generator.h5') #change according to you


In [9]:
# import cv2
# import matplotlib.pyplot as plt

# # Load the grayscale image
# image = cv2.imread('/kaggle/input/sar-s1-s2/dataset2/v_2/agri/s1/ROIs1868_summer_s1_59_p10.png', cv2.IMREAD_GRAYSCALE)

# # Apply a colormap


# # Display the colorized image using Matplotlib
# plt.imshow(cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB))
# plt.axis('off')
# plt.show()


NameError: name 'color_image' is not defined

In [10]:
# # Load and preprocess the grayscale image
# def preprocess_image(image_path, target_size=(256, 256)):
#     grayscale_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
#     grayscale_img = cv2.applyColorMap(grayscale_img, cv2.COLORMAP_VIRIDIS)
#     grayscale_img = cv2.resize(grayscale_img, target_size)  # Resize to match the model's input size
#     grayscale_img = grayscale_img / 255.0  # Normalize the image to [0, 1]
#     grayscale_img = np.expand_dims(grayscale_img, axis=-1)  # Add channel dimension (1 channel for grayscale)
#     grayscale_img = np.expand_dims(grayscale_img, axis=0)  # Add batch dimension
#     return grayscale_img

# # Example
# grayscale_image = preprocess_image('/kaggle/input/sar-s1-s2/dataset2/v_2/agri/s1/ROIs1868_summer_s1_59_p10.png')
# # Generate the colorized image using the generator
# colorized_image = generator.predict(grayscale_image)
# # Post-process the generated image
# def postprocess_image(colorized_image):
#     colorized_image = colorized_image[0]  # Remove batch dimension
#     colorized_image = (colorized_image * 255).astype(np.uint8)  # Rescale to [0, 255]
#     return colorized_image

# colorized_image = postprocess_image(colorized_image)

# # Save or display the colorized image
# cv2.imwrite('/kaggle/working/Model/colorized_image.png', cv2.cvtColor(colorized_image, cv2.COLOR_RGB2BGR))

# # Display the image
# plt.imshow(cv2.cvtColor(colorized_image, cv2.COLOR_BGR2RGB))
# plt.axis('off')
# plt.show()


ValueError: Exception encountered when calling Functional.call().

[1mInput 0 of layer "conv2d" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (1, 256, 256, 3)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(1, 256, 256, 3, 1), dtype=float32)
  • training=False
  • mask=None